### Vector Optimization 
suppose that we have the class Vertex that is defined as below:
```c++
class Vertex 
{
    int x, y, z; 
    Vertex(int a, int b, int c): x(a), y(b), z(c) {} 
    Vertex(const Vertex& other): x(other.x), y(other.y),z(other.z) {} 
};
```
if we run the main program with a vector of Vertices and we try to append new Vertices to the vector. 
```c++
int main() 
{
    std::vector<Vertex>vec; 
    vec.push_back(Vertex(1,2,3));
    vec.push_back(Vertex(2,3,4)); 
    vec.push_back(Vertex(3,4,5)); 
}
```
This will cause the program to call copy constructor 6 times. Copy constructors are called because objects are not directly allocated on the vector, but in main, and copied to vector. As for number 6, this is because everytime a `push_back` is called, the vector's memory allocation is not enough, so compiler has to reallocate memory, effectively redo the whole thing. We got 6 because from first line to last, we have 1, 2, 3 copies made, and 6 = 1 + 2 + 3. <br>
`push_back` has been a bottleneck in C++, so in C++ 11 and up, `emplace_back` is introduced to directly construct on the vector, and `reserve` can reserve memory space in advance. Different from the `std::vector<Type>(n)` constructor, which has to call the `Type` class's constructor, `reserve` only reserves memory required. 

```c++
int main() 
{
    std::vector<Vertex>vec;
    vec.reserve(3); // knowing that we need 3 
    vec.emplace_back(1,2,3);
    vec.emplace_back(2,3,4);
    vec.emplace_back(3,4,5); 
}
```
The code above will make no copies at all, and everything is directly constructed in the vector without being copied or reallocated. 


### Return Multiple Values from Function
One classic way to do this is to modify values in the _pass by reference_ arguments. This has good performance, but make get too messy with too many arguments. __not clean__.<br> 
Another way that yields the same principle, but a little bit more flexible, is to use pointer as arguments (which can also be modified in function body, and could be optional if we pass a nullptr). <br>
<br>
__Pair Return__
When returning two values, this is the standard method with the help of STL. creation of pair is via `std::make_pair` in the utility library  
```c++
#include <tuple>
#include <utility> 

static std::pair<std::string, int> Func(const std::string a, int b)
{
    // ... function body
    return std::make_pair<std::string, int>(a,b) 
}
```
for the caller, the elements in the pair can be obtained by:
```c++

auto result = Func("a",1);
string result_str = result.first;
int result_int = result.second; 
```
The `first, second` are not very clean. <br>

__Struct Return__
This makes the code more readable.
```c++
struct DemoStruct 
{
    std::string a;
    int b; 
};

static Func(const std::string a, int b)
{
    // ... function body
    return {a, b};   // this is the default constructor in C++11 
}

// in main 
auto result = Func("a", 1);
result_str = result.a; 
result_int = result.b; 
```

### Template Metaprogramming 


__General Usage: Generic Function__

```c++
template <typename T>
void Print(T value)
{
    std::cout<<value<<std::endl;
}


int main() 
{
    // type of the template function is decided at compile time
    // template specifies how to create methods 
    // methods automatically created at compile time 
    Print<std::string>("hello"); 
    Print<int>(3);
    Print(6);  
}

```


__Compile Time Variables__
when we create a struct in C++, and if we have an array declared inside, it has to be fixed size. However, with templates, it can be determined at compile time, without the use of dynamic arrays.

```c++
template <int SIZE>
class Array
{
private:
    int m_Array[SIZE]; 
public:
    int GetSize() const {return N;}
}

// in main 
// creation needs to specify the templated value 
Array<5> array; 
int a = array.GetSize(); 
```
Go even further to embed generic types 

```c++
template <typename T, int SIZE> 
class Array
{
private:
    T m_Array[SIZE]; 
public:
    int GetSize() const {return N;}
}

// in main
Array<std::string, 5> array; 
```



### _auto_ keyword

The `auto` keyword brings benefits in cases where we can simply determine the type at compile time. It is especially good when we are going through a complex STL container using iterators. In this case, `auto` can automatically determine the element type in the container. <br>
It is bad when we need to deal with the variable declared with `auto`. Since its type is undefined, no type-specific methods can be used on it. <br>
`auto` can also be used for lambda function closures. 
The main benefit is for __code readability__, or making the code cleaner. <br> 

__auto to handle messy types__ 

Now in the C++, dealing with a map 
```c++
class Device {}; 

class DeviceManager
{
private:
    std::unordered_map<std:string, std::vector<Device*>> m_Devices;
public:
    const std::unordered_map<std:string, std::vector<Device*>>& GetDevices() const; 
};

// in main 
DeviceManager dm; 
const auto& devices = dm.GetDevices(); 
```
<br>

__auto for iteration__
this is very similar to the duck-typed python, which has syntax like:
```python
for item in dictionary: 
    # do something 
```
Although c++ is in no way as easy as python, it can be done in this way:
```c++
std::map<std::wstring, std::map<std::wstring, int>> StudentGrades;

for (auto Map_Iter = StudentGrades.begin(); Map_Iter != StudentGrades.end(); ++Map_Iter) 
{
    // loop body 
}
```
_Range For loop_ is even better in C++11:

```c++
for (auto const &Iter : StudentGrades)
{
    // do something 
}
```
This is almost very close to the python way for iteration, a breeze! 
<br>
<br>
__Lambda Closure auto__ 
This makes reuse of lambda function possible 

```c++
auto func_multiply = [] (int a, int b) -> int { return a * b; }; 
std::cout << func_multiply(2,3)<<std::endl; 
```
It is argubly better than using a function object for the lambda expression, the function pointer way is:
```c++
std::function<int (int, int)> func_multiply = [] (int a, int b) -> int {return a * b;}; 
```

__auto keyword forces variable initialization__ failure to do so will not compile. 

<br>
#### Do not use the keyword if:
1. To intercept value of functions and then pass them as parameteres to another function 
```c++
auto a = Fun1();
Fun2(a); 
```

2. To capture result of an asynchronous operation
It is always better to be explicit about type in async. 

### Function Pointers 

when functions are not called, but are used for assignment, the assignment is actually a function pointer assignment. i.e. the variable on left hand side will receive the memory location of the function.

```c++
void func(int a) { 
    std::cout <<"hello"<<a<<std::endl; 
}

//in main 
void(*function)(int); // actual type, void(*)() function is a name 
auto function = func; // readability, simplest way

typedef void(*HelloFunction)(int); // explicit typedef clause 
HelloFunction foo = func; 
foo(); 
```
For each defined explicitly 
```c++
void ForEach(const std::vector<int>& values, void(*funct)(int))
{
    for (int value : values)
        funct(value);
}

// in main
std::vector<int> vec = {1, 2, 3, 4, 5}; 
ForEach(values, func); // like a mapping function that applies to each element 
```

A better way to handle this is clearly using the `for_each` function in std. This function supports both function pointers and lambda functions. 

```c++
// using our old method declared above
ForEach(values, [] (int value) {std::cout<<"Value:"<<value<<std::endl;}); 

// using the for_each method 
auto print = [] (const int& value) {std::cout<<"Values :"<<value<<std::endl;}; 
std::for_each(values.begin(), values.end(), print);
std::for_each(values.begin(), values.end(), [] (int &n) {n++;};

// operator struct/function object for_each 
struct Sum
{
    Sum(): sum(0) {}
    void operator() (int n) {sum+=n;}
    int sum;
};
// the function object is used, and function is called 
std::for_each(values.begin(), values.end(), Sum()); 
```



### Lambdas 
This is heavily used in languages such as python as Ruby, C++11 has great support for it too. 

```c++
#include <functional> 
// the lambda is of type std::function<void(int)>
void ForEach(const std::vector<int>& values, const std::function<void(int)>& func)
{
    for (int value: values)
        func(value);
}

// the bracket can be used to pass value in 

int a = 5; 
auto lambda = [a](int value) { std::cout<<"value"<<value<<" a: "<<a<<endl; }; 
ForEach(values, lambda); 


// for_each method 
std::for_each(values.begin(), values.end(), lambda); 
```

__find_if__

lambda functions are perfect for STL function like `find_if`, which prints out the first hit by the function pointer. 
```c++
std::vector<int> values = {1, 2, 3, 4, 5} 
std::find_if(values.begin(), values.end(), [](int value) {return value > 3;}; // return 4 

```

### Argmax, Argmin, Max, Min
```c++
int argMin = std::distance(values.begin(), std::min_element(x.begin(), x.end()));
int argMax = std::distance(values.begin(), std::max_element(x.begin(), x.end())); 
```
Thanks to the STL library updates in C++11, we now are able to write these in one line. 

In [ ]:
|